In [75]:
from random import seed
from random import randrange
from csv import reader

In [76]:
seed(1)

In [77]:
filename = 'data_banknote_authentication.txt'
file = open(filename, "r")
lines = reader(file)
dataset = list(lines)
dataset[0]

['3.6216', '8.6661', '-2.8073', '-0.44699', '0']

In [78]:
for i in range(len(dataset[0])):
    for row in dataset:
        row[i] = float(row[i].strip())

In [79]:
dataset[0]

[3.6216, 8.6661, -2.8073, -0.44699, 0.0]

In [80]:
n_folds = 5

# scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)

#k-fold


dataset_split = list()
dataset_copy = list(dataset)
fold_size = int(len(dataset) / n_folds)
for i in range(n_folds):
    fold = list()
    while len(fold) < fold_size:
        index = randrange(len(dataset_copy))
        fold.append(dataset_copy.pop(index))
    dataset_split.append(fold)
folds = dataset_split


In [89]:
class_values = list(set(row[-1] for row in dataset))
b_index, b_value, b_score, b_groups = 999, 999, 999, None
result = {'index':[], 'value':[], 'groups':[], 'gini_score':[]}
for index in range(len(dataset[0])-1):
    b_score = 999
    for row in dataset:
        left, right = list(), list()
        for row2 in dataset:
            if row2[index] < row[index]:
                left.append(row2)
            else:
                right.append(row2)
        groups = left,right
        gini = 0.0
        for class_value in class_values:
            for group in groups:
                size = len(group)
                if size == 0:
                    continue
                proportion = [row[-1] for row in group].count(class_value) / float(size)
                gini += (proportion * (1.0 - proportion))
        if gini < b_score:
            b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    result['index'].append(b_index)
    result['value'].append(b_value)
    result['groups'].append(b_groups)
    result['gini_score'].append(b_score)


In [106]:
print(result['gini_score'])
print(result['value'])

[0.45589336804211156, 0.48, 0.4811126069908904, 0.4938631012588292]
[1.7939, -6.9321, 8.8483, -8.5482]


In [99]:
root = {}
min_gini = min(result['gini_score'])
for i in range(len(result['gini_score'])):
    if result['gini_score'][i] == min_gini:
        ind = i
for key in result.keys():
    root[key] = result[key][ind]


# ^ Last edited

In [ ]:
depth = 1
max_depth = 5
min_size = 10
node = root
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)


def split(node, max_depth, min_size, depth):
left, right = node['groups']
del(node['groups'])
# check for a no split
if not left or not right:
    node['left'] = node['right'] = to_terminal(left + right)
    return
# check for max depth
if depth >= max_depth:
    node['left'], node['right'] = to_terminal(left), to_terminal(right)
    return
# process left child
if len(left) <= min_size:
    node['left'] = to_terminal(left)
else:
    node['left'] = get_split(left)
    split(node['left'], max_depth, min_size, depth+1)
# process right child
if len(right) <= min_size:
    node['right'] = to_terminal(right)
else:
    node['right'] = get_split(right)
    split(node['right'], max_depth, min_size, depth+1)


In [104]:
a = [1,2,3,4]
a.count()

TypeError: count() takes exactly one argument (0 given)

In [ ]:
scores = list()
for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set, [])
    test_set = list()
    for row in fold:
        row_copy = list(row)
        test_set.append(row_copy)
        row_copy[-1] = None
    predicted = algorithm(train_set, test_set, *args)
    actual = [row[-1] for row in fold]
    accuracy = accuracy_metric(actual, predicted)
    scores.append(accuracy)
